In [1]:
from PIL import Image
from IPython.display import display
import fitz
import numpy as np
from time import sleep
import sys

In [2]:
'''
A few different questions we want to ask about this pdf.
1. what parts are worth memorizing?
2. how do we know that?

(lara) i believe we should start out by listing the different sections of the textbook

1. Mate
2. Mate in 2
3. Mate in 3

4. 600 miniature games
5. Simple Endgames
6. Polgar Sisters' Tournament-Games Combinations (5207-5334)

7. [Just one problem - not sure why that's there]

8. Solutions

so one direction we could do is that we could convert this textbook into a nested dictionary, like

'''

textbook_dict = {
    
    "mate in one" : {},
    "mate in two" : {},
    "mate in three" : {},
    "600 miniature games" : {},
    "simple endgames" : {},
    "polgar sisters combinations" : {}
}

'''
and within each dictionary you have a string as a key, and a string as a value
both sides of the flashcard basically.
'''

'\nand within each dictionary you have a string as a key, and a string as a value\nboth sides of the flashcard basically.\n'

In [3]:
def merge_vertically(images):

    total_height = 0
    max_width = 0
    # find the width and height of the final image
    
    for img in images:
        total_height += img.size[1]
        max_width = max(max_width, img.size[0])
    
    new_img = Image.new('RGB', (max_width, total_height))
    # Write the contents of the new image
    current_height = 0
    for img in images:
      new_img.paste(img, (0,current_height))
      current_height += img.size[1]   
    
    return new_img

In [46]:
# we can start separating images into their own individual problems

def split_into_individual_problems(image_PIL, zoomfactor,problem_number):
    column_separation = 48 * zoomfactor
    
    # https://datacarpentry.org/image-processing/aio/index.html

    # convert each image into a numpy array
    # taken from here https://stackoverflow.com/questions/384759/how-to-convert-a-pil-image-into-a-numpy-array
    im2arr = np.array(image_PIL) # im2arr.shape: height x width x channel
    
    
    #arr2im = Image.fromarray(im2arr)    
    #print(type(im2arr))
    
    total_height, total_width = image_PIL.size[1], image_PIL.size[0]
    #print(str(total_width) + "  -  " + str(total_height))

    # the first line in the array that has more than 6 black pixels marks the beginning of the next problem
    # of course, you're going to want to subtract by like, 6 * zoomfactor to get the actual y-coordinate
    # add that value to a list
    problem_start_coords = []
    y = 1
    while True:
        if y >= total_height:
            break
        else:
            pass
                
        if im2arr[y][40*zoomfactor].all() == np.array([250, 250, 250]).all():
            y += 1
            continue
        else:
            #print(im2arr[y][40*zoomfactor])
            
            estimated_problem_start = y - (6 * zoomfactor)
            problem_start_coords.append(estimated_problem_start)
            
            y += 13 * zoomfactor
            #print(estimated_problem_start)
    
    # now, we have the pixel coordinates of each problem. we should start our extraction from the bottom-up
    problem_start_coords.reverse()
    
    total_problems_so_far = len(problem_start_coords) + problem_number - 1

    
    for problem_pixel_start in problem_start_coords:
        # extract the problem
        problem_image = image_PIL.crop((column_separation, problem_pixel_start, total_width, total_height))
        # problem_image.show()
        
        # and now we need to save that file
        problem_image.save('./individual_solutions/' + str(total_problems_so_far) + '.png',format="png")
        total_problems_so_far -= 1
        problem_number += 1
        
        # and then get a new cropped version of the image
        #print(total_height - problem_pixel_start)
        image_PIL = image_PIL.crop((0, 0, total_width, problem_pixel_start))
        
        # and then we get the new height (width stays the same)
        total_height = image_PIL.size[1]
        
    
    return problem_number

    
    

In [ ]:
'''
there are exactly 5334 problems. what we can do is start off \
with obtaining the solutions and getting them into their own dictionary.
'''

solutions = {}

# the first step is to extract the pdf pages from the textbook that *only* contain answers.
# specifically, the pages are 972 to 1178

doc = fitz.open("23.pdf")

current_count = 972 - 1
#current_count = 1166
problem_number = 1
while True:
    page = doc.load_page(current_count)
    current_count += 1
    
    # this gives us a temp.png
    zoomfactor = 6
    pix = page.getPixmap(matrix=fitz.Matrix(zoomfactor, zoomfactor))
    output = "temp.png"
    pix.writePNG(output)
    
    # now we need to trip the header of that temp image!
    # Opens a image in RGB mode
    im = Image.open("temp.png")

    # Setting the points for cropped image
    left = 0 * zoomfactor
    top = 72 * zoomfactor
    right = 600 * zoomfactor
    bottom = 950 * zoomfactor

    # Cropped image of above dimension
    # (It will not change original image)
    im1 = im.crop((left, top, right, bottom))

    # Shows the image in image viewer
    #im1.show()
    
    # we're going to have whatever is on the left side
    left_side = im1.crop((0*zoomfactor,0*zoomfactor,300*zoomfactor,(bottom - top)))
    #left_side.show()
    #display(left_side)
    
    # and whatever is on the right size!
    right_side = im1.crop(((290+17)*zoomfactor,0*zoomfactor,600*zoomfactor,(bottom - top)))
    #right_side.show()
    
    # merge them together vertically
    left_top_right_down = merge_vertically([left_side,right_side])
    #left_top_right_down.show()
    to_add = left_top_right_down.crop((30*zoomfactor,0,290*zoomfactor,(bottom - top)*2))
    #to_add.show()
    #to_add.save('./stacked_images/' + str(current_count) + '.png',format="png")
    
    #problemcount += split_into_individual_problems(to_add,zoomfactor)
    #print(problemcount)
    problem_number = split_into_individual_problems(to_add,zoomfactor,problem_number)
    if current_count == 1178: 
        print("done")
        break
    else: pass



In [45]:
# first we should probably figure out the page offsets 
# as in, the real page number for Chapter One: Mates is 16
# the book says that the page number is actually nine!

#pdf_offset = 7

# hm the offset might not remain the same. let's just do it manually

chapters = {
    1 : {"chapter_start" : 9 + 7},
    2 : {"chapter_start" : 62 + 7},
    3 : {"chapter_start" : 636 + 4},
    4 : {"chapter_start" : 819 + 2},
    5 : {"chapter_start" : 921 + 1},
    6 : {"chapter_start" : 949 - 2},
    7 : {"chapter_start" : 971 - 1}    
}

# of course, we need to figure out where each chapter ends.
chapter_ends = [66,637,763,920,945,968,971]

for chapter_number in chapters:
    chapters[chapter_number]['chapter_end'] = chapter_ends[chapter_number - 1]

print(chapters)

'''
now, the format of each section that we've selected is the same.
1. six problems per page max
2. same headers
3. two columns per page

so we can reuse past code to split each into individual problems.
'''

{1: {'chapter_start': 16, 'chapter_end': 66}, 2: {'chapter_start': 69, 'chapter_end': 637}, 3: {'chapter_start': 640, 'chapter_end': 763}, 4: {'chapter_start': 821, 'chapter_end': 920}, 5: {'chapter_start': 922, 'chapter_end': 945}, 6: {'chapter_start': 947, 'chapter_end': 968}, 7: {'chapter_start': 970, 'chapter_end': 971}}


In [44]:
# now that we have a folder of problems that have been relatively cleaned up,

# there's also another thing hanging over our heads: what should we memorize? 
# should we just turn the entire thing into cards?
# do we use anki?
